# FID & KID computation

## Copy 10k CHOLECT45 data from https://github.com/CAMMA-public/cholect45 to dest_path. 
### Comment out cells below if you need to prepare a folder with real data.
### Copy with margin becasue some frames are all black. Exclude videos 01, 49, 25, 66, 52, 56.  

In [ ]:
# import os, shutil
# import random 
# import glob
# from tqdm import tqdm
# import random
# import os
# from PIL import Image
# from torchvision.transforms import functional as F
# from torchvision.transforms import InterpolationMode
# import torchvision.transforms as transforms

In [1]:
# n=10500 #draw more to exclude all black frames

# cholect_path = '/path/to/real_data'
# dest_path = 'dest/path/to/real_data'
# os.makedirs(dest_path, exist_ok=True)

# items=[vid for vid in os.listdir(cholect_path) if all(t not in vid for t in ['01', '49', '25', '66', '52', '56']) ]

# for i in tqdm(items):
#     imgs = [img for img in os.listdir(os.path.join(cholect_path, i)) if ".png" in img]
#     frames_to_sample = n//len(items)
#     imgs = random.sample(imgs, frames_to_sample+1)
#     for img in imgs:
#         src_path = os.path.join(cholect_path, i, img)
#         out_path = os.path.join(dest_path, str(i +"_" + img))
#         shutil.copy(src_path, out_path)

100%|██████████| 39/39 [01:28<00:00,  2.26s/it]


### Remove black frames

In [3]:

# def is_almost_black(image_path, threshold=10):
#     """
#     Check if an image is almost black by comparing pixel values with a threshold.
#     """
#     image = Image.open(image_path).convert('L')  # Open the image and convert it to grayscale
#     pixels = image.getdata()
#     for pixel in pixels:
#         if pixel > threshold:
#             return False
#     return True

# def remove_black_images(folder_path):
#     """
#     Remove black and almost black images from a folder.
#     """
#     for filename in tqdm(os.listdir(folder_path)):
#         if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust the file extensions as needed
#             image_path = os.path.join(folder_path, filename)
#             if is_almost_black(image_path):
#                 os.remove(image_path)

# # Example usage
# remove_black_images(dest_path)


100%|██████████| 10530/10530 [01:37<00:00, 107.88it/s]


### Remove excessive images to keep equal 10k needed for FID, KID

In [4]:
# file_list = os.listdir(dest_path)
# files_to_delete = random.sample(file_list, len(file_list) - 10000)

# for file_name in files_to_delete:
#     file_path = os.path.join(dest_path, file_name)
#     os.remove(file_path)


### Resize and crop images, save to separate folder

In [8]:
# dest_path_cropped = dest_path +"_cropped"

# os.makedirs(folder_cropped, exist_ok=True)

# for filename in tqdm.tqdm(os.listdir(dest_path)):
#     if filename.endswith('.jpeg') or filename.endswith('.png') or filename.endswith('.jpg'):
#         # Open the image file
#         image = Image.open(os.path.join(dest_path, filename)).convert("RGB")
#         image = transforms.Resize(299)(image)
#         image = transforms.CenterCrop(299)(image)
#         # Save the cropped image with the same name as the original
#         image.save(os.path.join(dest_path_cropped, filename))

100%|██████████| 10000/10000 [06:53<00:00, 24.20it/s]


## Calculate FID and KID 
### We first resize synthetic images to 299 and copy them to tmp folder - we always do it that way to have unified pipeline (resizing impacts fid & kid values, we want to have full control).

In [ ]:
# !pip install torch-fidelity

In [ ]:
import shutil
import os
import tqdm
from PIL import Image
from torchvision import transforms
import random
import glob
import torch_fidelity

In [ ]:
real_data_path = 'path/to/real/data/cropped' # dest_path_cropped from previous cells
data_path_25_66 = '/path/to/data' #output_vid25_66
data_path_01_49 = '/path/to/data' #output_vid01_49
data_path_52_56 = '/path/to/data' #output_vid52_56

# calculate FID & KID for mixed style
rand_dict = {1: data_path_25_66,
            2: data_path_01_49,
            3: data_path_52_56}

filenames_common =set(os.listdir(rand_dict[1]))& set(os.listdir(rand_dict[2])) & set(os.listdir(rand_dict[3]))
filenames = [os.path.join(rand_dict[random.randint(1, 3)], f) for f in filenames_common]

random.seed(420420)
filenames = random.sample(filenames,10000)

In [ ]:
# save images in a tmp folder. It is important to keep exactly the same resizing method.
tmp_folder = "/path/to/tmp/folder"

if os.path.exists(tmp_folder):
    shutil.rmtree(tmp_folder)
os.makedirs(tmp_folder, exist_ok=True)

for filename in tqdm.tqdm(filenames):
    if filename.endswith('.jpeg') or filename.endswith('.png') or filename.endswith('.jpg'):
        # Open the image file
        image = Image.open(os.path.join(filename)).convert("RGB")
        image = transforms.Resize(299)(image)
        # Save the cropped image with the same name as the original
        image.save(os.path.join(tmp_folder, os.path.basename(filename)))

In [ ]:
# calculate FID and KID
metrics_dict = torch_fidelity.calculate_metrics(
    input1=real_data_path,
    input2=tmp_folder, 
    cuda=True, 
    isc=False, 
    fid=True, 
    kid=True, 
    prc=False, 
    verbose=True,
    kid_subset_size =1000,
    cache = False
)

